In [6]:
# !pip install fasttext

In [26]:
import os
import time
import csv
import numpy as np
import pandas as pd
import fasttext
import gensim
import sklearn.metrics
import re
import unicodedata
from tqdm.auto import tqdm
import sys
import hashlib
import random

train_file = 'nlbse23-issue-classification-train.csv'
eval_file = 'nlbse23-issue-classification-eval.csv'

csv.field_size_limit(sys.maxsize) # to avoid error: _csv.Error: field larger than field limit (131072)

9223372036854775807

In [18]:
def csv_rows(filename):
	with open(filename, "r", newline='', encoding='utf-8') as f:
		return sum(1 for _ in csv.DictReader(f))

# download the training set if it does not exist
# if not os.path.isfile("github-labels-top3-803k-train.csv"):
#   !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-train.tar.gz" | tar -xz

display(pd.read_csv(train_file, nrows=5))

# if not os.path.isfile("github-labels-top3-803k-test.csv"):
#   !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-test.tar.gz" | tar -xz

display(pd.read_csv(eval_file, nrows=5))

,id,labels,title,body,author_association
0,1199051804,documentation,setting a logging Handler name,BPO | [43058](https://bugs.python.org/issue430...,MANNEQUIN
1,1199074324,documentation,Improve documentation for typing._GenericAlias,BPO | [46589](https://bugs.python.org/issue465...,MANNEQUIN
2,1199022454,documentation,Description of '\w' behavior is vague in `re` ...,BPO | [38566](https://bugs.python.org/issue385...,MANNEQUIN
3,1199028356,documentation,add docstrings to functions in pdb module,BPO | [39278](https://bugs.python.org/issue392...,MANNEQUIN
4,1199055394,documentation,Documentation needs to declare CalledProcessEr...,BPO | [43635](https://bugs.python.org/issue436...,MANNEQUIN


,id,labels,title,body,author_association
0,1199053386,documentation,A possible misleading expression in the Virtua...,BPO | [43319](https://bugs.python.org/issue433...,MANNEQUIN
1,1255069635,bug,[BUG] a valid `gameName` in the `create a new ...,**Describe the bug**\r\nIn the `create a new p...,NONE
2,1089772715,feature,How to check if a certain entity still exists?,During a bug in my own code I noticed that the...,NONE
3,1000928729,feature,chose the timezone in dbeaver option,"Dbeaver 21.2.0\r\n\r\nFor all version DBeaver,...",NONE
4,1300011093,bug,[Issue]: Multiple Versions of a Movie not work...,### Please describe your bug\n\nThe doc at htt...,NONE


In [12]:

def count_tokens(text):
	return text.count(" ") + 1

def preprocess(text):
  text = str(text)

  text = text.replace("__label__", "")

  # lowercase
  # text = text.lower()
  
  # remove html tags
  # text = gensim.parsing.preprocessing.strip_tags(text)
  
  # remove punctuation
  # text = gensim.parsing.preprocessing.strip_punctuation(text)
  
  # remove numerics
  # text = gensim.parsing.preprocessing.strip_numeric(text)
  
  # remove consecutive whitespace characters and convert tabs to spaces
  text = gensim.parsing.preprocessing.strip_multiple_whitespaces(text)
  
  # text = gensim.parsing.preprocessing.strip_short(text, minsize=3)
  
  # text = gensim.parsing.preprocessing.remove_stopwords(text)
  
  # text = gensim.parsing.preprocessing.stem_text(text)
  
  return text

def preprocess_row(row):
  doc = row["author_association"].lower()
  doc += " " + preprocess(row["title"])
  doc += " " + preprocess(row["body"])

  return doc

In [40]:

def transform_to_fasttext_format(i_path, o_path):
	with open(i_path, "r", newline='', encoding="utf-8") as i_f, open(o_path, "w", encoding='utf-8') as o_f:
		reader = csv.DictReader(i_f)
		for row in tqdm(reader, desc="Transform to fastText format", total=csv_rows(i_path)):
			o_f.write(f"__label__{row['labels']} {preprocess_row(row)}\n")

transform_to_fasttext_format(train_file, "issues.train")

!wc -l issues.train
!head -n 2 issues.train

Transform to fastText format:   0%|          | 0/1275881 [00:00<?, ?it/s]

1275881 issues.train
__label__documentation mannequin setting a logging Handler name BPO | [43058](https://bugs.python.org/issue43058) --- | :--- Nosy | @vsajip <sup>*Note: these values reflect the state of the issue at the time it was migrated and might not reflect the current state.*</sup> <details><summary>Show more details</summary><p> GitHub fields: ```python assignee = None closed_at = None created_at = <Date 2021-01-28.20:17:02.239> labels = ['3.8', 'type-feature', 'library', 'docs'] title = 'setting a logging Handler name' updated_at = <Date 2021-01-28.20:17:02.239> user = 'https://bugs.python.org/bcohen' ``` bugs.python.org fields: ```python activity = <Date 2021-01-28.20:17:02.239> actor = 'bcohen' assignee = 'docs@python' closed = False closed_date = None closer = None components = ['Documentation', 'Library (Lib)'] creation = <Date 2021-01-28.20:17:02.239> creator = 'bcohen' dependencies = [] files = [] hgrepos = [] issue_num = 43058 keywords = [] message_count = 1.0 messag

In [84]:
# https://fasttext.cc/docs/en/python-module.html#train_supervised-parameters
model = fasttext.train_supervised("issues.train")
model.save_model(f"issues.bin")

# model.quantize()
# model.save_model(f"issues.ftz")

Read 171M words
Number of words:  11143356
Number of labels: 4
Progress: 100.0% words/sec/thread: 1690300 lr:  0.000000 avg.loss:  0.456748 ETA:   0h 0m 0s 14.8% words/sec/thread: 1689055 lr:  0.085224 avg.loss:  0.550585 ETA:   0h 0m39s 22.4% words/sec/thread: 1690013 lr:  0.077628 avg.loss:  0.531281 ETA:   0h 0m35s 25.4% words/sec/thread: 1689950 lr:  0.074593 avg.loss:  0.528451 ETA:   0h 0m34s 44.1% words/sec/thread: 1690707 lr:  0.055930 avg.loss:  0.508727 ETA:   0h 0m25s 45.1% words/sec/thread: 1690503 lr:  0.054852 avg.loss:  0.506000 ETA:   0h 0m25s 54.0% words/sec/thread: 1690447 lr:  0.045962 avg.loss:  0.500313 ETA:   0h 0m21s 61.6% words/sec/thread: 1690349 lr:  0.038376 avg.loss:  0.493090 ETA:   0h 0m17s 62.7% words/sec/thread: 1690396 lr:  0.037290 avg.loss:  0.493159 ETA:   0h 0m17s 94.4% words/sec/thread: 1690324 lr:  0.005635 avg.loss:  0.459913 ETA:   0h 0m 2s 96.1% words/sec/thread: 1690258 lr:  0.003904 avg.loss:  0.459503 ETA:   0h 0m 1s 97.8% words/sec/thread: 

In [85]:
# confusion matrix
y_true = []
y_pred = []

with open(eval_file, newline='', encoding='utf-8') as f:
  reader = csv.DictReader(f)
  for row in tqdm(reader, desc="Benchmarking Inference Performance"):
    pred = model.predict(preprocess_row(row))[0][0][9:]
    y_true.append(row["labels"])
    y_pred.append(pred)

for label in ["bug", "feature", "question", "documentation"]:
  P_c = sklearn.metrics.precision_score(y_true, y_pred, average=None, labels=[label])[0]
  R_c = sklearn.metrics.recall_score(y_true, y_pred, average=None, labels=[label])[0]
  F1_c = sklearn.metrics.f1_score(y_true, y_pred, average=None, labels=[label])[0]
  print(f"=*= {label} =*=")
  print(f"precision:\t{P_c:.4f}")
  print(f"recall:\t\t{R_c:.4f}")
  print(f"F1 score:\t{F1_c:.4f}")
  print()


P = sklearn.metrics.precision_score(y_true, y_pred, average='micro')
R = sklearn.metrics.recall_score(y_true, y_pred, average='micro')
F1 = sklearn.metrics.f1_score(y_true, y_pred, average='micro')

print("=*= global =*=")
print(f"precision:\t{P:.4f}")
print(f"recall:\t\t{R:.4f}")
print(f"F1 score:\t{F1:.4f}")

Benchmarking Inference Performance: 0it [00:00, ?it/s]

=*= bug =*=
precision:	0.8766
recall:		0.9180
F1 score:	0.8968

=*= feature =*=
precision:	0.8443
recall:		0.8612
F1 score:	0.8527

=*= question =*=
precision:	0.6699
recall:		0.4670
F1 score:	0.5504

=*= documentation =*=
precision:	0.7411
recall:		0.5022
F1 score:	0.5987

=*= global =*=
precision:	0.8518
recall:		0.8518
F1 score:	0.8518
